# 增加Epoch

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [3]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\\KaiKeBa\\基础班\Python\\第六章\\第四节\\4-CNN(2)\\data\\hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=1)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 244
     Root location: E:\KaiKeBa\基础班\Python\第六章\第四节\4-CNN(2)\data\hymenoptera_data\train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ), 'val': Dataset ImageFolder
     Number of datapoints: 153
     Root location: E:\KaiKeBa\基础班\Python\第六章\第四节\4-CNN(2)\data\hymenoptera_data\val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=PIL.Image.BILINEAR)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [5]:
dataset_sizes
class_names

{'train': 244, 'val': 153}

['ants', 'bees']

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Finetuning the convnet

In [7]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [8]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### epoch=10

In [9]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------


D:\Anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.6412 Acc: 0.6967
val Loss: 0.2169 Acc: 0.9085

Epoch 1/9
----------
train Loss: 0.4326 Acc: 0.8115
val Loss: 0.3838 Acc: 0.8301

Epoch 2/9
----------
train Loss: 0.6373 Acc: 0.7746
val Loss: 0.3057 Acc: 0.8562

Epoch 3/9
----------
train Loss: 0.4768 Acc: 0.7828
val Loss: 0.2147 Acc: 0.9216

Epoch 4/9
----------
train Loss: 0.5189 Acc: 0.7910
val Loss: 0.3608 Acc: 0.8889

Epoch 5/9
----------
train Loss: 0.5129 Acc: 0.8197
val Loss: 0.3630 Acc: 0.8693

Epoch 6/9
----------
train Loss: 0.4064 Acc: 0.8238
val Loss: 0.3101 Acc: 0.8954

Epoch 7/9
----------
train Loss: 0.3648 Acc: 0.8156
val Loss: 0.3116 Acc: 0.8824

Epoch 8/9
----------
train Loss: 0.3309 Acc: 0.8443
val Loss: 0.2520 Acc: 0.9216

Epoch 9/9
----------
train Loss: 0.3089 Acc: 0.8566
val Loss: 0.2364 Acc: 0.9412

Training complete in 1m 25s
Best val Acc: 0.941176


### epoch=50

In [10]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=50)

Epoch 0/49
----------
train Loss: 0.2153 Acc: 0.9098
val Loss: 0.2537 Acc: 0.9216

Epoch 1/49
----------
train Loss: 0.3738 Acc: 0.8197
val Loss: 0.3248 Acc: 0.8627

Epoch 2/49
----------
train Loss: 0.2924 Acc: 0.8607
val Loss: 0.2411 Acc: 0.9150

Epoch 3/49
----------
train Loss: 0.2683 Acc: 0.8852
val Loss: 0.2444 Acc: 0.9085

Epoch 4/49
----------
train Loss: 0.2894 Acc: 0.8811
val Loss: 0.2620 Acc: 0.9085

Epoch 5/49
----------
train Loss: 0.3936 Acc: 0.8279
val Loss: 0.2543 Acc: 0.9020

Epoch 6/49
----------
train Loss: 0.2746 Acc: 0.9057
val Loss: 0.2362 Acc: 0.9216

Epoch 7/49
----------
train Loss: 0.2547 Acc: 0.8934
val Loss: 0.2184 Acc: 0.9281

Epoch 8/49
----------
train Loss: 0.2854 Acc: 0.8811
val Loss: 0.2431 Acc: 0.9150

Epoch 9/49
----------
train Loss: 0.2828 Acc: 0.8770
val Loss: 0.2476 Acc: 0.9020

Epoch 10/49
----------
train Loss: 0.2679 Acc: 0.8852
val Loss: 0.2597 Acc: 0.9020

Epoch 11/49
----------
train Loss: 0.3056 Acc: 0.8770
val Loss: 0.2134 Acc: 0.9216

Ep

## ConvNet as fixed feature extractor

In [9]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
    
# Parameters of newly sconstructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_con = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_con, step_size=7, gamma=0.1)

In [10]:
model_conv

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### epoch=10

In [11]:
model_conv = train_model(model_conv, criterion, optimizer_con,exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------


D:\Anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.6749 Acc: 0.6066
val Loss: 0.2208 Acc: 0.9477

Epoch 1/9
----------
train Loss: 0.4536 Acc: 0.7951
val Loss: 0.3589 Acc: 0.8562

Epoch 2/9
----------
train Loss: 0.7742 Acc: 0.7459
val Loss: 0.2561 Acc: 0.9216

Epoch 3/9
----------
train Loss: 0.3820 Acc: 0.8525
val Loss: 0.2004 Acc: 0.9346

Epoch 4/9
----------
train Loss: 0.4657 Acc: 0.8074
val Loss: 0.5854 Acc: 0.7974

Epoch 5/9
----------
train Loss: 0.7410 Acc: 0.7336
val Loss: 0.1940 Acc: 0.9346

Epoch 6/9
----------
train Loss: 0.4309 Acc: 0.8197
val Loss: 0.1962 Acc: 0.9346

Epoch 7/9
----------
train Loss: 0.3587 Acc: 0.8361
val Loss: 0.1849 Acc: 0.9412

Epoch 8/9
----------
train Loss: 0.3456 Acc: 0.8484
val Loss: 0.1936 Acc: 0.9346

Epoch 9/9
----------
train Loss: 0.2985 Acc: 0.8811
val Loss: 0.1747 Acc: 0.9412

Training complete in 0m 39s
Best val Acc: 0.947712


### epoch=50

In [12]:
model_conv = train_model(model_conv, criterion, optimizer_con,exp_lr_scheduler, num_epochs=50)

Epoch 0/49
----------
train Loss: 0.3943 Acc: 0.8156
val Loss: 0.2285 Acc: 0.9477

Epoch 1/49
----------
train Loss: 0.3427 Acc: 0.8566
val Loss: 0.2071 Acc: 0.9477

Epoch 2/49
----------
train Loss: 0.4110 Acc: 0.8074
val Loss: 0.2114 Acc: 0.9412

Epoch 3/49
----------
train Loss: 0.3267 Acc: 0.8770
val Loss: 0.2135 Acc: 0.9542

Epoch 4/49
----------
train Loss: 0.3730 Acc: 0.7992
val Loss: 0.2107 Acc: 0.9477

Epoch 5/49
----------
train Loss: 0.3431 Acc: 0.8361
val Loss: 0.2137 Acc: 0.9542

Epoch 6/49
----------
train Loss: 0.3788 Acc: 0.8197
val Loss: 0.2103 Acc: 0.9477

Epoch 7/49
----------
train Loss: 0.3742 Acc: 0.8361
val Loss: 0.2111 Acc: 0.9477

Epoch 8/49
----------
train Loss: 0.3465 Acc: 0.8443
val Loss: 0.2259 Acc: 0.9477

Epoch 9/49
----------
train Loss: 0.3773 Acc: 0.8279
val Loss: 0.2049 Acc: 0.9477

Epoch 10/49
----------
train Loss: 0.3533 Acc: 0.8525
val Loss: 0.2095 Acc: 0.9412

Epoch 11/49
----------
train Loss: 0.4060 Acc: 0.8238
val Loss: 0.2063 Acc: 0.9477

Ep

In [13]:
model_conv = train_model(model_conv, criterion, optimizer_con,exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.3805 Acc: 0.8279
val Loss: 0.2151 Acc: 0.9477

Epoch 1/24
----------
train Loss: 0.3540 Acc: 0.8320
val Loss: 0.2159 Acc: 0.9346

Epoch 2/24
----------
train Loss: 0.3493 Acc: 0.8361
val Loss: 0.2070 Acc: 0.9477

Epoch 3/24
----------
train Loss: 0.4117 Acc: 0.7869
val Loss: 0.2064 Acc: 0.9477

Epoch 4/24
----------
train Loss: 0.3885 Acc: 0.8238
val Loss: 0.2127 Acc: 0.9477

Epoch 5/24
----------
train Loss: 0.3945 Acc: 0.8197
val Loss: 0.2230 Acc: 0.9477

Epoch 6/24
----------
train Loss: 0.4338 Acc: 0.7951
val Loss: 0.2101 Acc: 0.9477

Epoch 7/24
----------
train Loss: 0.4030 Acc: 0.8033
val Loss: 0.2022 Acc: 0.9477

Epoch 8/24
----------
train Loss: 0.4067 Acc: 0.8320
val Loss: 0.2028 Acc: 0.9477

Epoch 9/24
----------
train Loss: 0.3571 Acc: 0.8566
val Loss: 0.2050 Acc: 0.9477

Epoch 10/24
----------
train Loss: 0.3276 Acc: 0.8648
val Loss: 0.2051 Acc: 0.9477

Epoch 11/24
----------
train Loss: 0.3345 Acc: 0.8689
val Loss: 0.2034 Acc: 0.9477

Ep